In [2]:
import torch
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np

# # Importing the project from a subfolder
sys.path.append('./project')

from models.transformer import Aggregator
from models.tokenizer import Tokenizer

ModuleNotFoundError: No module named 'models'

# Test the Aggregator

In [2]:
test_model = Aggregator(embed_dim=64)

In [15]:
B, S, P, C = 5, 3, 20, 64 # B events, S images per event, P tokens per image, C elements per token

device = "cuda" if torch.cuda.is_available() else "cpu"

test_input = torch.tensor(np.random.randn(B, S, P, C)).float().to(device)
test_pos = torch.tensor(np.random.randint(1, 8, size=(B, S, P, 2))).to(device)
test_model = test_model.to(device)

In [20]:
test_output, test_idx = test_model.forward(test_input, test_pos)

In [23]:
test_output[0].shape

torch.Size([5, 3, 25, 128])

# Test the tokenizer

In [6]:
test_model = Tokenizer()

In [7]:
B, H = 5, 16 # B total patches, each image a HxH square

device = "cuda" if torch.cuda.is_available() else "cpu"

test_input = torch.tensor(np.random.randn(B, H, H)).float().to(device)
test_input = test_input.view(B, 1, H, H)
test_model = test_model.to(device)

In [8]:
test_output = test_model.forward(test_input)

In [10]:
test_output.shape

torch.Size([5, 256, 1, 1])

In [11]:
test_output.view(5, -1).shape

torch.Size([5, 256])

# Test both

In [2]:
test_tokenizer = Tokenizer()
test_aggregator = Aggregator(embed_dim=256)

In [9]:
B, S, P, H = 1, 3, 5, 16 # B events, S images per event, P patches per image, HxH patches

device = "cuda" if torch.cuda.is_available() else "cpu"

test_input = torch.tensor(np.random.randn(B, S, P, H, H)).float().to(device)
test_pos = torch.tensor(np.random.randint(1, H+1, size=(B, S, P, 2))).to(device)
test_tokenizer, test_aggregator = test_tokenizer.to(device), test_aggregator.to(device)

In [14]:
# Simple flattening for this test. In the real case this would involve recording the sequence lengths
test_tokens = test_tokenizer.forward(test_input.view(B*S*P, H, H)).view(B, S, P, 256)
test_output, test_idx = test_aggregator.forward(test_tokens, test_pos)

In [16]:
test_output[0].shape

torch.Size([1, 3, 10, 512])